<a href="https://colab.research.google.com/github/gakutakimoto/gaku-lala-apps/blob/main/M_Tracer_Analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 📌 必要なライブラリをインポート
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# 📌 データを読み込む（Colabにアップロード済みのファイルを使う）
file_path = "/content/sample_data/swing_detail.csv"
data = pd.read_csv(file_path, dtype=str, low_memory=False)

# 📌 数値カラムを適切な型に変換
num_cols = [
    'impactClubPath', 'impactHandFirst', 'downSwingShaftRotationMin',
    'impactAttackAngle', 'topFaceAngleToHorizontal', 'impactGripSpeed',
    'addressHandFirst', 'addressLieAngle', 'impactLieAngle', 'profileHeight', 'impactHeadSpeed'
]
data[num_cols] = data[num_cols].astype(float)

# 📌 使用するカラム & 重み設定
feature_columns = {
    'impactClubPath': 2.0,  # 最重要（×2.0）
    'impactHandFirst': 1.5,  # 重要（×1.5）
    'downSwingShaftRotationMin': 1.5,  # 重要（×1.5）
    'impactAttackAngle': 1.0,  # 補助（×1.0）
    'topFaceAngleToHorizontal': 1.0,  # 補助（×1.0）
    'impactGripSpeed': 1.0,  # 補助（×1.0）
    'addressHandFirst': 1.0,  # 補助（×1.0）
    'addressLieAngle': 1.0,  # 補助（×1.0）
    'impactLieAngle': 1.0   # 補助（×1.0）
}

# 📌 必要なカラムだけ抽出
data_selected = data[list(feature_columns.keys())]

# 📌 欠損値を処理（平均値で補完）
data_selected = data_selected.fillna(data_selected.mean())

# 📌 データの標準化（スケール統一）
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_selected)

# 📌 重み付けを適用
weights = np.array(list(feature_columns.values()))
data_scaled_weighted = data_scaled * weights

# 📌 クラスタリング実施（クラスタ数 = 5）
kmeans = KMeans(n_clusters=5, random_state=42, n_init=1)  # n_initを1にして計算負荷を軽減
data_selected['cluster'] = kmeans.fit_predict(data_scaled_weighted)

# 📌 元データにクラスタ情報を追加
data_final = data.merge(data_selected[['cluster']], left_index=True, right_index=True)

# 📌 クラスタ名の辞書
cluster_names = {
    0: "アウトサイドイン・コントロール型",
    1: "ストレート・パワーヒッター型",
    2: "インサイドアウト・フェード型",
    3: "インサイドアウト・ドローヒッター型",
    4: "ストレート・アスリート型"
}

# 📌 身長カテゴリ（170cm 以上 / 未満を 1 / 0 に変換）
data_final['height_category'] = np.where(data_final['profileHeight'] >= 170, "1", "0")

# 📌 ヘッドスピードカテゴリを作成（Slow / Mid / Fast → 0, 1, 2 に変換）
bins = [0, 30, 37, 100]
labels = ['0', '1', '2']  # SlowHS → 0, MidHS → 1, FastHS → 2
data_final['hs_category'] = pd.cut(
    data_final['impactHeadSpeed'], bins=bins, labels=labels, include_lowest=True
).astype(str)  # 🛠 Categorical → 文字列に変換

# 📌 クラスタ名（スイングタイプ）
data_final['swing_type'] = data_final['cluster'].map(cluster_names)

# 📌 クラスタIDの修正（数値化したヘッドスピードを使用）
data_final['cluster_id'] = (
    data_final['cluster'].astype(str) + '-' +
    data_final['height_category'] + '-' +
    data_final['hs_category']
)

# 📌 最終スインググループ名
data_final['final_swing_group'] = (
    data_final['swing_type'].astype(str) + '-' +
    data_final['height_category'].astype(str) + '-' +
    data_final['hs_category']
)

# ✅ 確認用の出力
display(data_final[['swing_type', 'height_category', 'hs_category', 'cluster_id', 'final_swing_group']].head(20))

# 📌 クラスタリング結果をCSVで保存（次回以降はこれを使う！）
csv_output_path = "/content/swing_cluster_results.csv"
data_final.to_csv(csv_output_path, index=False)
print(f"✅ クラスタリング結果をCSVに保存しました: {csv_output_path}")

# 📌 クラスタリング結果をExcelで保存
excel_output_path = "/content/swing_cluster_results.xlsx"
data_final.to_excel(excel_output_path, index=False)
print(f"✅ クラスタリング結果をExcelに保存しました: {excel_output_path}")


,swing_type,height_category,hs_category,cluster_id,final_swing_group
0,インサイドアウト・フェード型,1,1,2-1-1,インサイドアウト・フェード型-1-1
1,ストレート・アスリート型,1,1,4-1-1,ストレート・アスリート型-1-1
2,ストレート・パワーヒッター型,0,2,1-0-2,ストレート・パワーヒッター型-0-2
3,インサイドアウト・フェード型,0,1,2-0-1,インサイドアウト・フェード型-0-1
4,インサイドアウト・フェード型,1,1,2-1-1,インサイドアウト・フェード型-1-1
5,ストレート・パワーヒッター型,0,2,1-0-2,ストレート・パワーヒッター型-0-2
6,インサイドアウト・フェード型,1,1,2-1-1,インサイドアウト・フェード型-1-1
7,ストレート・パワーヒッター型,0,2,1-0-2,ストレート・パワーヒッター型-0-2
8,インサイドアウト・フェード型,1,1,2-1-1,インサイドアウト・フェード型-1-1
9,ストレート・パワーヒッター型,0,2,1-0-2,ストレート・パワーヒッター型-0-2


✅ クラスタリング結果をCSVに保存しました: /content/swing_cluster_results.csv


In [2]:
import pandas as pd

# 📌 クラスタリング済みデータをロード（最初の1000行だけ読む）
file_path = "/content/swing_cluster_results.csv"
df = pd.read_csv(file_path, nrows=1000)  # 🔥 メモリ負荷を減らす

# 📌 先頭5行を表示（確認用）
display(df.head(5))

import pandas as pd

# 📌 CSVファイルのパス
file_path = "/content/swing_cluster_results.csv"

# 📌 まずは最初の1000行だけ読む（メモリ節約）
df = pd.read_csv(file_path, nrows=1000)

# 📌 先頭5行を表示（データ確認）
display(df.head())

# 📌 行数とカラム情報を確認
print(f"✅ 総行数: {len(df)}")
print(f"✅ カラム一覧: {df.columns.tolist()}")


,id,swingInfoId,swingDate,swingDateAccuracy,dst,userId,profileBirthYear,profileBirthMonth,profileBirthDay,profileGender,...,analyzeResultType,isExistVideo,isFavorite,memo,cluster,height_category,hs_category,swing_type,cluster_id,final_swing_group
0,1,c95ac17af35fde5b777424bc11e3d8a8,2024-12-28T11:53:46Z,1,0,fe5179f0e72a4c649b0b337df44c46ce,1900,1,1,0,...,0,0,0,NaN,2,1,1,インサイドアウト・フェード型,2-1-1,インサイドアウト・フェード型-1-1
1,2,6fcfbf797af4dfa74dd8cf294b0e6c0c,2024-12-28T11:52:41Z,1,0,fe5179f0e72a4c649b0b337df44c46ce,1900,1,1,0,...,0,0,0,NaN,4,1,1,ストレート・アスリート型,4-1-1,ストレート・アスリート型-1-1
2,3,bc73a6943ce2bfcbd925b46b232bd06d,2024-12-28T11:48:11Z,1,0,a60ba835ba564e0e88cc807c17c16acb,1970,6,24,0,...,0,0,0,NaN,1,0,2,ストレート・パワーヒッター型,1-0-2,ストレート・パワーヒッター型-0-2
3,4,59eec859a580438876ab2ee4d49526a2,2024-12-28T11:47:24Z,1,0,a60ba835ba564e0e88cc807c17c16acb,1970,6,24,0,...,0,0,0,NaN,2,0,1,インサイドアウト・フェード型,2-0-1,インサイドアウト・フェード型-0-1
4,5,c8b1a33602deb426a68e7f1dbe3a238e,2024-12-28T11:47:20Z,1,0,fe5179f0e72a4c649b0b337df44c46ce,1900,1,1,0,...,0,0,0,NaN,2,1,1,インサイドアウト・フェード型,2-1-1,インサイドアウト・フェード型-1-1


,id,swingInfoId,swingDate,swingDateAccuracy,dst,userId,profileBirthYear,profileBirthMonth,profileBirthDay,profileGender,...,analyzeResultType,isExistVideo,isFavorite,memo,cluster,height_category,hs_category,swing_type,cluster_id,final_swing_group
0,1,c95ac17af35fde5b777424bc11e3d8a8,2024-12-28T11:53:46Z,1,0,fe5179f0e72a4c649b0b337df44c46ce,1900,1,1,0,...,0,0,0,NaN,2,1,1,インサイドアウト・フェード型,2-1-1,インサイドアウト・フェード型-1-1
1,2,6fcfbf797af4dfa74dd8cf294b0e6c0c,2024-12-28T11:52:41Z,1,0,fe5179f0e72a4c649b0b337df44c46ce,1900,1,1,0,...,0,0,0,NaN,4,1,1,ストレート・アスリート型,4-1-1,ストレート・アスリート型-1-1
2,3,bc73a6943ce2bfcbd925b46b232bd06d,2024-12-28T11:48:11Z,1,0,a60ba835ba564e0e88cc807c17c16acb,1970,6,24,0,...,0,0,0,NaN,1,0,2,ストレート・パワーヒッター型,1-0-2,ストレート・パワーヒッター型-0-2
3,4,59eec859a580438876ab2ee4d49526a2,2024-12-28T11:47:24Z,1,0,a60ba835ba564e0e88cc807c17c16acb,1970,6,24,0,...,0,0,0,NaN,2,0,1,インサイドアウト・フェード型,2-0-1,インサイドアウト・フェード型-0-1
4,5,c8b1a33602deb426a68e7f1dbe3a238e,2024-12-28T11:47:20Z,1,0,fe5179f0e72a4c649b0b337df44c46ce,1900,1,1,0,...,0,0,0,NaN,2,1,1,インサイドアウト・フェード型,2-1-1,インサイドアウト・フェード型-1-1


✅ 総行数: 1000
✅ カラム一覧: ['id', 'swingInfoId', 'swingDate', 'swingDateAccuracy', 'dst', 'userId', 'profileBirthYear', 'profileBirthMonth', 'profileBirthDay', 'profileGender', 'profileHeight', 'clubLength', 'clubFaceAngle', 'clubLoftAngle', 'clubLieAngle', 'real', 'clubShaftHardness', 'clubMakerId', 'clubId', 'fwMajorVersion', 'fwMinorVersion', 'fwPatchVersion', 'algMajorVersion', 'algMinorVersion', 'swingType', 'isBroken', 'serialNo', 'dataVersion', 'timeZoneOffset', 'gpsLongitude', 'gpsAltitude', 'gpsLatitude', 'addressHandFirst', 'addressLieAngle', 'impactClubPath', 'impactHandFirst', 'impactAttackAngle', 'impactHeadSpeed', 'impactLieAngle', 'impactRelativeFaceAngle', 'impactPointX', 'impactShaftRotation', 'impactPointY', 'impactFaceAngle', 'impactGripSpeed', 'impactLoftAngle', 'maxHeadSpeed', 'maxGripSpeed', 'topFaceAngleToHorizontal', 'halfwaydownFaceAngleToVertical', 'naturalReleaseTiming', 'swingTempo', 'downSwingShaftRotationMax', 'downSwingShaftRotationMin', 'halfwaybackFaceAngleTo